In [1]:
import numpy as np
import cv2
import sklearn
import pickle

In [2]:
# face detection
face_detector_model = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt', 
                                               './models/res10_300x300_ssd_iter_140000_fp16.caffemodel')


# feature extraction
face_feature_model = cv2.dnn.readNetFromTorch('./models/openface.nn4.small2.v1.t7')


# face recognition
face_recognition_model = pickle.load(open('./models/ml_face_person_identity.pkl', mode='rb'))

In [7]:
# pipeline models

img = cv2.imread('./images/1.jpeg')

image = img.copy()
h,w = img.shape[:2]
img_blob = cv2.dnn.blobFromImage(img, 1, (300,300), (104,177,123), swapRB=False, crop=False)
face_detector_model.setInput(img_blob)
detections = face_detector_model.forward()

if len(detections) > 0:
    for i, confidence in enumerate(detections[0,0,:,2]):
        if confidence > 0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            startx, starty, endx, endy = box.astype('int')
            cv2.rectangle(image, (startx, starty),(endx, endy), (0,255,0))
            
            # feature extraction
            face_roi = img[starty:endy, startx:endx]
            face_blob = cv2.dnn.blobFromImage(face_roi,1/255,(96,96), (0,0,0),swapRB=True, crop=True)
            face_feature_model.setInput(face_blob)
            verctors = face_feature_model.forward()
            
            # predict with machine learning
            face_name = face_recognition_model.predict(verctors)
            face_score = face_recognition_model.predict_proba(verctors).max()
            
            print(face_name)
            print(face_score)


cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

['ronaldo']
0.5888964668153781
